### Made by Carlos Gonçalves, nr.13071

In [2]:
using ExcelReaders
using DataFrames
using Query;

# Data

In [2]:
df = readxl(DataFrame, "/Users/Dino/Desktop/pwt90.xlsx", "Data!A1:AU11831");

In [118]:
showcols(df)

11830×47 DataFrames.DataFrame
│ Col # │ Name          │ Eltype  │ Missing │
├───────┼───────────────┼─────────┼─────────┤
│ 1     │ countrycode   │ String  │ 0       │
│ 2     │ country       │ String  │ 0       │
│ 3     │ currency_unit │ String  │ 0       │
│ 4     │ year          │ Float64 │ 0       │
│ 5     │ rgdpe         │ Float64 │ 2391    │
│ 6     │ rgdpo         │ Float64 │ 2391    │
│ 7     │ pop           │ Float64 │ 2391    │
│ 8     │ emp           │ Float64 │ 3586    │
│ 9     │ avh           │ Float64 │ 8511    │
│ 10    │ hc            │ Float64 │ 3963    │
│ 11    │ ccon          │ Float64 │ 2391    │
│ 12    │ cda           │ Float64 │ 2391    │
│ 13    │ cgdpe         │ Float64 │ 2391    │
│ 14    │ cgdpo         │ Float64 │ 2391    │
│ 15    │ ck            │ Float64 │ 2421    │
│ 16    │ ctfp          │ Float64 │ 5877    │
│ 17    │ cwtfp         │ Float64 │ 5877    │
│ 18    │ rgdpna        │ Float64 │ 2391    │
│ 19    │ rconna        │ Float64 │ 2391    │
│ 20

<br>

# Question1

In [4]:
data1 = @from i in df begin
     @where (i.year == 2014 || i.year == 2000) && (i.countrycode == "PRT" || i.countrycode == "THA")
     @select {i.country, i.year, i.rgdpo, i.emp, i.rkna, i.labsh}
     @collect DataFrame
end

,country,year,rgdpo,emp,rkna,labsh
1,Portugal,2000.0,219630.03125,5.0140275955200195,1.30302975e6,0.6547812819480896
2,Portugal,2014.0,281354.90625,4.3413472175598145,1.7131235e6,0.5807642936706543
3,Thailand,2000.0,434126.28125,31.74068260192871,2.38961475e6,0.38971802592277527
4,Thailand,2014.0,929554.5625,38.893306732177734,3.62297275e6,0.3928701877593994



|      |    Portugal|    | Thailand  | |
| ------------- |:-------------:||:-------------:|  -----:|
|   year   | 2010 | 2014           | 2010  | 2014 |
|   rgdpo   | 219630.03125 | 281354.90625           | 434126.28125  | 929554.5625 |
|  emp    | 5.0140275955200195 | 4.3413472175598145           | 31.74068260192871  | 38.893306732177734 |
| rkna     | 1.30302975e6 | 1.7131235e6           | 2.38961475e6  | 3.62297275e6 |
| labsh     | 0.6547812819480896 | 0.5807642936706543           | 0.38971802592277527  | 0.3928701877593994 |

<br>

# Question2

In [157]:
function ex2()
    gdp = []
    for i in 1:2 push!(gdp, ((data1[i*2,3]/data1[i*2-1,3])^(1/(data1[i*2,2]-data1[i*2-1,2]))-1)*100) end
    results = ["PRT" "THA"; gdp[1] gdp[2]]
    return results
end;

In [158]:
ex2()

2×2 Array{Any,2}:
  "PRT"    "THA" 
 1.78483  5.58895

|      | Portugal           | Thailand  |
| ------------- |:-------------:| -----:|
| GDP growth rate     | 1.78483% | 5.588954% |

<br>

# Question3

In [161]:
function ex3()
    results = ["Country" "TFP2000" "TFP2014" "ΔTFP/TFP"]
    for i in 1:2
        α = 1 - data1[i*2-1,6]
        TFP2000 = data1[i*2-1,3]/(data1[i*2-1,5]^α * data1[i*2-1,4]^(1-α))
        TFP2014 = data1[i*2,3]/(data1[i*2,5]^α * data1[i*2,4]^(1-α))
        results = [results; data1[i*2,1] TFP2000 TFP2014 (TFP2014-TFP2000)/TFP2000*100]
    end
    return results
end;

In [162]:
ex3()

3×4 Array{Any,2}:
 "Country"      "TFP2000"     "TFP2014"    "ΔTFP/TFP"
 "Portugal"  591.869       758.105       28.0865     
 "Thailand"   14.4488       22.1714      53.4483     


|      | Portugal |       | Thailand|  |
| ------------- |:-------------:||:-------------:|  -----:|
|   TFP2000   | 2010 | 2014           | 2010  | 2014 |
|   TFP2014   |  591.869  | 758.105           | 14.4488  | 22.1714 |
|  ΔTFP/TFP (%)    |  | 28.0865%           |   | 53.4483% |


<br>

# Question4

$$ Y = A \cdot K^\alpha \cdot L^{1-\alpha} $$

$$ \frac{\Delta Y}{Y} = \frac{\Delta A}{A} + \alpha \cdot \frac{\Delta K}{K} + (1-\alpha) \cdot  \frac{\Delta L}{L} $$

In [114]:
function ex4()
    changes = ["Country" "ΔY/Y" "ΔTFP/TFP" "α*ΔK/K" "(1-α)*ΔL/L"]
    contributions = ["Country" "Contribution TFP" "Contribution K" "Contribution L"]
    for i in 1:2
        α = 1 - data1[i*2-1,6]
        ΔY_Y = (data1[i*2,3]-data1[i*2-1,3])/data1[i*2-1,3]
        ΔA_A = ex3()[i+1,4]
        ΔK_K = (data1[i*2,5]-data1[i*2-1,5])/data1[i*2-1,5]
        ΔL_L = (data1[i*2,4]-data1[i*2-1,4])/data1[i*2-1,4]
        deltas = [ΔA_A α*ΔK_K (1-α)*ΔL_L]
        changes = [changes; data1[i*2,1] ΔY_Y  deltas]
        contributions = [contributions; data1[i*2,1] deltas*(100/ΔY_Y)]
    end
    return changes, contributions
end;

In [115]:
ex4()[1]

3×5 Array{Any,2}:
 "Country"    "ΔY/Y"   "ΔTFP/TFP"   "α*ΔK/K"    "(1-α)*ΔL/L"
 "Portugal"  0.28104  0.280865     0.108648   -0.0878453    
 "Thailand"  1.14121  0.534483     0.314986    0.0878213    

|   Country   | ΔY/Y |    ΔTFP/TFP   | α*ΔK/K|  (1-α)*ΔL/L|
| ------------- |:-------------:||:-------------:|  -----:|
|  Portugal   | 0.28104 | 0.280865           | 0.108648  | -0.0878453 |
|   Thailand  |  1.14121  | 0.534483           | 0.314986  | 0.0878213 |

In [116]:
ex4()[2]

3×4 Array{Any,2}:
 "Country"     "Contribution TFP"    "Contribution K"     "Contribution L"
 "Portugal"  99.9378               38.6594             -31.2572           
 "Thailand"  46.8349               27.6011               7.69547          

|   Country   | Contribution TFP, (%) |    Contribution K, (%)   | Contribution L, (%)|
| ------------- |:-------------:|  -----:|
|  Portugal   | 99.9378 | 38.6594            | -31.2572  |
|   Thailand  |  46.8349  | 27.6011           | 7.69547 |

<br>

Answer: 